<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-Data-Loading" data-toc-modified-id="Imports-and-Data-Loading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and Data Loading</a></span><ul class="toc-item"><li><span><a href="#DataLoader-class" data-toc-modified-id="DataLoader-class-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><a href="https://pytorch.org/docs/stable/data.html" target="_blank">DataLoader class</a></a></span></li></ul></li><li><span><a href="#Building-a-Neural--Network" data-toc-modified-id="Building-a-Neural--Network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Building a Neural  Network</a></span><ul class="toc-item"><li><span><a href="#Activation-Functions" data-toc-modified-id="Activation-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Activation Functions</a></span></li><li><span><a href="#Loss-Function" data-toc-modified-id="Loss-Function-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Loss Function</a></span></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="#Optimization" data-toc-modified-id="Optimization-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Optimization</a></span></li><li><span><a href="#GPU" data-toc-modified-id="GPU-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>GPU</a></span></li><li><span><a href="#Training-Loop" data-toc-modified-id="Training-Loop-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Training Loop</a></span></li></ul></li><li><span><a href="#Predictions" data-toc-modified-id="Predictions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Predictions</a></span></li><li><span><a href="#Saving-and-Loading-the-Model" data-toc-modified-id="Saving-and-Loading-the-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Saving and Loading the Model</a></span></li><li><span><a href="#References" data-toc-modified-id="References-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>References</a></span></li><li><span><a href="#Next-Steps:" data-toc-modified-id="Next-Steps:-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Next Steps:</a></span><ul class="toc-item"><li><span><a href="#(Computer-Vision)-Cat-Dog" data-toc-modified-id="(Computer-Vision)-Cat-Dog-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>(Computer Vision) Cat Dog</a></span></li></ul></li></ul></div>

## Imports and Data Loading

In [2]:
# import PyTorch
import torch

# standard DS stack
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
# embed static images in the ipynb
%matplotlib inline 

# neural network package
import torch.nn as nn 
import torch.nn.functional as F

# computer vision
import torchvision
from torchvision import transforms
from PIL import Image

# dataset loading
from torch.utils.data import Dataset, DataLoader, ConcatDataset

import copy
# import tqdm

### [DataLoader class](https://pytorch.org/docs/stable/data.html)

Every dataset, no matter whether what it includes, can interact with PyTorch if it satisfies the following abstract Python class:

```python
class Dataset(object):
    def __getitem__(self, idx):
        """ Retrieve an item from the dataset in a (label, tensor) pair.
        
        Args:
            idx: index
        """
        pass
        
    def __len__(self):
        """ Returns the size of the dataset (len)"""
        pass
```

This is referred to as a map-style dataset in the docs. 

## Building a Neural  Network

In [3]:
class Net(nn.Module): # class inherits from nn.Module
    def __init__(self):
        super(Net, self).__init__() # initialize nn.Module
        # some fully connected layers w/ linear transformation
        """ nn.Linear(in_features, out_features, bias=True)
        Args:
            in_features: size of each input sample. For input shape (28, 28), 
                we would have in_features = 28 * 28 = 784
            out_features: size of each output sample.
        """
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 3)
        self.fc3 = nn.Linear(3, 1)
    def forward(self, x): # defines the forward propagation
        x = F.relu(self.fc1(x)) # relu activation function
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        # Output layer needs a multiclassifying transformation
        # log softmax works for this
        return F.log_softmax(x, dim=1)
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
)


This above network is known as a **feedforward neural network** or **multilayer perception (MLP)**. It's called feedforward because there are no **feedback** connections in which the outputs of the model are fed back into previous layers.  

When feedforward neural networks are extended to include feedback connections, they are called **recurrent neural netowrks**. 

The **depth** of a network is defined as its number of layers (including the output layer but excluding input layer), while the **width** of a network is defined to be the maximal number of nodes in a layer. This explains the reasoning behind the name, "deep learning".

The terminology for the network structure above is typical called the **network architecture**, which includes how many layers the network contains, how the layers are connected to each other, and how many neurons (a.k.a nodes, a.k.a. units) are in each layer.

### Activation Functions

Activation functions are used to compute hidden layer values.

Here, we make use of the **rectified linear unit (ReLU)** as the activation function, using `F.relu`. This is the default recommendation for the activation function in modern deep learning. Here's [an article](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/) about ReLU for your reference. 

### Loss Function 

TODO: Explain loss function. 

Training a network involves passing data through the network, using the loss function to ~~determine~~ define a criterion for capturing the similarity or difference between a prediction and an actual target.

TODO: Implement loss function

TODO: Explain choice of loss function. 

## Training 

### Optimization

The information about the loss function that is gained when we pass data through the network is used to update the weights of the network such that we  minimize the loss function.

In order to perform the updates on the neural network, we use an optimizer. 

In [6]:
# Implement Adam optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

The **learning rate**, `lr`, is often a key parameter that needs to be tweaked in order to get a network to learn properly and efficiently.

Adaptive moment estimation (Adam) and stochastic gradient descent (SGD) have been empirically shown to outperform most other optimizers in deep learning networks. 

I decided to use Adam for this tutorial because it, along with RMSProp and AdaGrad, uses an adaptive learning rate, which adapts its updates to each paramter depending on the importance of individual paramters. 

### GPU

PyTorch, by default, does CPU-based calculations. To take advantage of the GPU, the input tensors and model need to be moved to the GPU explicitly with the `to()` method.

`network` is simply an instance of the neural network class written above. 

In [4]:
# GPU Recipe:
if torch.cuda.is_available(): # If PyTorch reports that GPU is available
    device = torch.device("cuda") # device = GPU
else:
    device = torch.device("cpu") # device = CPU

network = net
network.to(device) # Copy model to device

Net(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
)

### Training Loop

TODO: Explain backpropagation at high level.

TODO: Add practical explanation to code. 

In [ ]:
def train(network, optimizer=optimizer, loss_fn, train_loader, val_loader,
          n_epochs, device=device):
    for epoch in range(n_epochs):
        train_loss = 0.0
        val_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, target = batch
            # transfer batch data to 
            inputs = inputs.to(device)
            target = target.to(device)
            output = network(inputs)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item()
        training_losss /= len(train_iterator)
    pass

## Predictions 

## Saving and Loading the Model 

## References
- Goodfellow, I., Bengio, Y., Courville, A., & Bengio, Y. (2016). *Deep learning* (Vol. 1). Cambridge: MIT press.
- Lu, Z., Pu, H., Wang, F., Hu, Z., & Wang, L. (2017). *The expressive power of neural networks: A view from the width*. In Advances in neural information processing systems (pp. 6231-6239).
- Brownlee, J. (2019). A gentle introduction to the rectified linear unit (relu). *Machine Learning Mastery. https://machinelearningmastery.com/rectified-linear-activation-function-fordeep-learning-neural-networks*.


----
## Next Steps:
### (Computer Vision) Cat Dog
  - Save a CNN and use it on this dataset. 
  - Explain fundamental CNN concepts. 
  - Data from [Kaggle competition](https://www.kaggle.com/c/dogs-vs-cats/overview)
  - [jaeboklee](https://www.kaggle.com/jaeboklee/pytorch-cat-vs-dog)